# Machine Learning Notebook for bike/parking space predictions

## Section 0: Import Required Libraries

In [33]:
from sqlalchemy import create_engine
from sklearn.metrics import accuracy_score, mean_squared_error
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
import pickle

## Section 1: Data preparation

### Section 1.1: Create a function to create to the SQL database

In [2]:
def connect_to_database():
    """Function for connecting to the SQL database"""

    # Create variables to store cretentials
    USER = "admin"
    DB = "dbikes"
    PORT = "3306"
    URL = "database-1.ctesjcult8dm.eu-west-1.rds.amazonaws.com"

    # Read in password from text file
    with open('../mysql_password.txt') as f:
        PASSWORD = ''.join(f.readlines())
        PASSWORD = str(PASSWORD).split()[0]

    # Create engine using credentials
    engine = create_engine(
        "mysql+mysqlconnector://{}:{}@{}:{}/{}".format(USER, PASSWORD, URL, PORT, DB), echo=True)

    # Create connection using engine and return connection
    conn = engine.connect()
    return conn


In [3]:
engine = connect_to_database()

2022-03-24 09:07:08,917 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2022-03-24 09:07:08,919 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-24 09:07:08,953 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2022-03-24 09:07:08,956 INFO sqlalchemy.engine.Engine [generated in 0.00878s] {}
2022-03-24 09:07:08,984 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-03-24 09:07:08,986 INFO sqlalchemy.engine.Engine [raw sql] {}


### Section 1.2: Read in real time bike/parking space availability data

In [4]:
availability_df = pd.read_sql_query("SELECT * FROM dbikes.availability", engine)

2022-03-24 09:07:09,107 INFO sqlalchemy.engine.Engine SELECT * FROM dbikes.availability
2022-03-24 09:07:09,109 INFO sqlalchemy.engine.Engine [raw sql] {}


### Section 1.3: Remove Duplicate values from real time availability

In [5]:
# Remove duplicates
availability_df.drop_duplicates(inplace=True)

### Section 1.4: Read in weather data

In [6]:
weather_df = pd.read_sql_query("SELECT dt, temperature, main FROM dbikes.real_time_weather", engine)

2022-03-24 09:07:37,438 INFO sqlalchemy.engine.Engine SELECT dt, temperature, main FROM dbikes.real_time_weather
2022-03-24 09:07:37,439 INFO sqlalchemy.engine.Engine [raw sql] {}


### Section 1.5: Remove duplicate values from weather

In [7]:
#Drop duplicates
weather_df.drop_duplicates(inplace=True)

### Section 1.6: Rename last update column in availability dataframe to dt to help with the merging of the availability and weather dataframes

In [8]:
#Changes the name of the column
availability_df.rename(columns={"last_update": "dt"}, inplace=True)
availability_df.columns

Index(['number', 'available_bikes', 'available_stands', 'dt'], dtype='object')

### Section 1.7: Change the type of the dt variable to an np.int64 type for merging

In [9]:
availability_df["dt"] = availability_df["dt"].astype(np.int64)
weather_df["dt"]= weather_df["dt"].astype(np.int64)

### Section 1.8: Sort the dt columns for merging

In [10]:
availability_df.sort_values(by="dt", inplace=True)
weather_df.sort_values(by="dt", inplace=True)

### Section 1.9: Convert seconds to milliseconds in dt column in availability so that it is comparable with the corresponding weather column

In [11]:
#Converts seconds to miliseconds
availability_df["dt"] = availability_df["dt"] // 1000

### Section 1.10: Merge weather and availability dataframes

In [12]:
df=pd.merge_asof(availability_df, weather_df, on="dt", direction="nearest")

### Section 1.11: Convert temperature from Kelvin to Celcius

In [13]:
df["temperature"] = df["temperature"] - 273.15

### Section 1.12: Transform the dt column to get hour of the day and day of the week

In [14]:
df["dt"] = pd.to_datetime(df["dt"], unit="s")
df["day_of_week"] = df["dt"].dt.day_name()
df["hour"] = df["dt"].dt.hour

### Section 1.13: Convert categorical variables to "category" data type

In [15]:
df["main"] = df["main"].astype("category")
df["day_of_week"] = df["day_of_week"].astype("category")
df["hour"] = df["hour"].astype("category")

### Section 1.14: Get a list of station numbers

In [16]:
station_numbers = list(df["number"].unique())

### Section 1.15: Reencode categorical variables using binary encoding

In [17]:
dummy_fields = ["main", "hour", "day_of_week"]
for each in dummy_fields:
    dummies = pd.get_dummies(df[each], prefix=each, drop_first = False)
    df = pd.concat([df, dummies], axis=1)
df = df.drop(dummy_fields, axis=1)

In [18]:
x_column_names = list(df.columns)[4:]

## Section 2: Model creation/evaluation

### Section 2.1: Comparison of different ML models on a sample of the data

We decided to compare a linear and logistic regression model on 20 of our bike stations, to try to ascertain if one model tends to outperform the other.

In [35]:
linear_regression = {}
logistic_regression = {}
linear_accuracy = {}
logistic_accuracy = {}

for station in station_numbers[:20]:
    # Prepare the training and test data for both linear and logistic regression
    station_df = df[df["number"] == station]
    X = station_df[x_column_names]
    y = station_df["available_bikes"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    # Fit a logistic regression model
    logistic_regression[station] = LogisticRegression( solver='lbfgs', max_iter=1500)
    logistic_regression[station].fit(X_train, y_train)
    y_pred_logistic = logistic_regression[station].predict(X_test)
    logistic_accuracy[station] = mean_squared_error(y_test, y_pred_logistic)

    # Fit a linear regression model
    linear_regression[station] = LinearRegression()
    linear_regression[station].fit(X_train, y_train)
    y_pred_linear = linear_regression[station].predict(X_test)
    linear_accuracy[station] = mean_squared_error(y_test, y_pred_linear)
    
    # file_name = "station_" + str(station) + "_bike_model.pkl"
    # with open(file_name, 'wb') as handle:
    #     pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)
    print(station)

print(linear_accuracy)
print(logistic_accuracy)

86
55
80
32
99
115
9
18
39


C:\Users\micha\anaconda3\envs\comp30830\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


13
62
117
42
12
82
107
28
65
40
95
{86: 64.74978649766668, 55: 18.20268286040057, 80: 57.741650911648414, 32: 39.881743838544736, 99: 18.978742062430353, 115: 32.925932491296614, 9: 32.726720392810336, 18: 32.266173314113246, 39: 19.814176111451168, 13: 27.807582091478118, 62: 29.6997877379184, 117: 43.92040831670165, 42: 33.12999781004087, 12: 19.919616724234004, 82: 21.960619635947907, 107: 50.695613446075065, 28: 34.4402953160158, 65: 69.23921796075479, 40: 33.30803209931803, 95: 73.28148788482353}
{86: 88.03798100949525, 55: 26.94009216589862, 80: 96.47985537190083, 32: 63.51328553353016, 99: 24.489951113525258, 115: 48.79770444763271, 9: 54.747770246164826, 18: 41.220479302832246, 39: 28.724420190995907, 13: 40.61331038439472, 62: 44.853240492769146, 117: 64.52163742690058, 42: 50.021872265966756, 12: 33.89886425094646, 82: 37.86682107701216, 107: 81.44715025906736, 28: 49.91372351160444, 65: 123.2880658436214, 40: 60.372191011235955, 95: 116.98347521281923}


#### Results

We see that linear regression tends to outperform logistic regression. Therefore, we have selected linear regression as our optimal model

### Section 2.2: Implementation of linear regression model

In [45]:
bike_availability = {}
station_availability = {}

for station in station_numbers:
    # Prepare the training and test data for both linear and logistic regression
    station_df = df[df["number"] == station]
    X = station_df[x_column_names]
    y_bikes = station_df["available_bikes"]
    y_stations = station_df["available_stands"]
    X_train_bikes, X_test_bikes, y_train_bikes, y_test_bikes = train_test_split(X, y_bikes, test_size=0.3)
    X_train_stations, X_test_stations, y_train_stations, y_test_stations = train_test_split(X, y_stations, test_size=0.3)

    # Fit a linear regression model for bike availability
    bike_availability[station] = LinearRegression()
    bike_availability[station].fit(X_train_bikes, y_train_bikes)

    # Fit a linear regression model for bike availability
    station_availability[station] = LinearRegression()
    station_availability[station].fit(X_train_stations, y_train_stations)
    
    file_name_bikes = "station_" + str(station) + "_bike_model.pkl"
    with open(file_name_bikes, 'wb') as handle:
        pickle.dump(bike_availability[station], handle, pickle.HIGHEST_PROTOCOL)

    file_name_stations = "station_" + str(station) + "_station_model.pkl"
    with open(file_name_stations, 'wb') as handle:
        pickle.dump(station_availability, handle, pickle.HIGHEST_PROTOCOL)

    print(station)

0.16562335195596833
86
0.3468532672018232
55
0.2851236714758697
80
0.3961294730276673
32
0.5015255312464701
99
0.5716900044478127
115
0.34137670318309377
9
0.22246940657948777
18
0.19200485103780718
39
0.31464845780196116
13
0.44687807819119485
62
0.248199576523905
117
0.33244371277814166
42
0.16058545703704263
12
0.3071509749406345
82
0.25591982239597666
107
0.1590579094439779
28
0.08274604171717814
65
0.13514889294822496
40
0.2035491305867403
95
0.10459712400954346
76
0.29280923215884014
47
0.6186539556494146
114
0.19535050679774857
15
0.0486972635523053
75
0.20886682056987627
24
0.12996773025005226
116
0.3033391160893142
25
0.23668118083965695
26
0.0648301808652183
16
0.1269035166176996
102
0.14719984306431233
59
0.14017807182828856
57
0.5591311248496381
112
0.12328450805951141
64
0.3441287113344287
3
0.10459844810373387
106
0.3584415310301039
109
0.29951559519218873
101
0.3300278865219749
21
0.5466220426474904
11
0.427883502596221
60
0.11187300890134733
58
0.25080766515030695
93
0.

## Section 3: Testing our final model

In [37]:
sample_x_data = X_test.head(1)
sample_x_data

,temperature,main_Clear,main_Clouds,main_Drizzle,main_Mist,main_Rain,main_Snow,hour_0,hour_1,hour_2,...,hour_21,hour_22,hour_23,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
444937,3.85,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [40]:
with open('station_32_bike_model.pkl', 'rb') as handle:
    station_32_model = pickle.load(handle)

In [43]:
print(station_32_model.predict(sample_x_data))

[6.86816406]
